In [2]:
# first cell, after switching kernels
try:
    spark.stop()
except:
    pass

from pyspark.sql import SparkSession
import os

spark = (
    SparkSession.builder
    .appName("spark-on-312")
    .master(os.getenv("SPARK_MASTER", "spark://spark-master:7077"))
    .config("spark.driver.bindAddress", "0.0.0.0")
    .config("spark.driver.host", "spark-notebooks")   # must equal your jupyter container_name
    .config("spark.driver.port", "7078")
    .config("spark.blockManager.port", "7079")
    # executors = Bitnami's Python 3.12
    .config("spark.executorEnv.PYSPARK_PYTHON", "/opt/bitnami/python/bin/python")
    .config("spark.pyspark.python", "/opt/bitnami/python/bin/python")
    .getOrCreate()
)


In [3]:
import sys, os
print("Driver Python:", sys.version)

def probe(_):
    import sys, os
    return (os.environ.get("PYSPARK_PYTHON"), sys.version, sys.executable)

print("Executor (PYSPARK_PYTHON, version, executable):",
      spark.sparkContext.parallelize([0], 1).map(probe).first())


Driver Python: 3.12.11 | packaged by conda-forge | (main, Jun  4 2025, 14:45:31) [GCC 13.3.0]
Executor (PYSPARK_PYTHON, version, executable): ('/opt/bitnami/python/bin/python', '3.12.11 (main, Aug  5 2025, 17:17:50) [GCC 12.2.0]', '/opt/bitnami/python/bin/python')


In [4]:
df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("../../../data/airlines_flights_data.csv")
)

df.printSchema()
df.show(5, truncate=False)

root
 |-- index: integer (nullable = true)
 |-- airline: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- source_city: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- stops: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- destination_city: string (nullable = true)
 |-- class: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- days_left: integer (nullable = true)
 |-- price: integer (nullable = true)

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index|airline |flight |source_city|departure_time|stops|arrival_time |destination_city|class  |duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|0    |SpiceJet|SG-8709|Delhi      |Evening       |zero |Night        |Mumbai          |Economy|2.17    |1        |5953 |
|1    |SpiceJet|

In [5]:
sc = spark.sparkContext

rdd = sc.textFile("/data/u.data")

ratings = rdd.map(lambda line: line.split("\t")[2])

counts = ratings.countByValue()

for k in sorted(counts, key=int):
    print(f"{k} {counts[k]}")

1 6110
2 11370
3 27145
4 34174
5 21201


In [5]:
import sys
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

print("Driver Python:", sys.version)
def pyver(_):
    import sys
    return sys.version
print("Executor Python:", spark.sparkContext.parallelize([0], 1).map(pyver).first())

Driver Python: 3.11.10 | packaged by conda-forge | (main, Oct 16 2024, 01:27:36) [GCC 13.3.0]


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 4 times, most recent failure: Lost task 0.3 in stage 8.0 (TID 26) (172.18.0.3 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1100, in main
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 12) than that in driver 3.11, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:784)
	at org.apache.spark.api.pyth